In [ ]:
from os import listdir
import pyart
from matplotlib import pyplot as plt
from matplotlib import colors as pltcolors
import matplotlib as mpl
from cartopy import crs as ccrs
from cartopy import feature as cfeat
from metpy.plots import USCOUNTIES
import xarray as xr
import numpy as np
from pyxlma.coords import RadarCoordinateSystem
import pandas as pd
from scipy import stats

In [ ]:
files = sorted(listdir('rax'))
fileds = ['rax/'+f for f in files if f.endswith('-D.nc')]
fileps = ['rax/'+f for f in files if f.endswith('-P.nc')]
filers = ['rax/'+f for f in files if f.endswith('-R.nc')]
filevs = ['rax/'+f for f in files if f.endswith('-V.nc')]
filews = ['rax/'+f for f in files if f.endswith('-W.nc')]
filezs = ['rax/'+f for f in files if f.endswith('-Z.nc')]


plasma = plt.cm.plasma_r(np.linspace(.05, .75, 50))
viridis = plt.cm.viridis(np.linspace(0, 1, 950))
plasmidis = pltcolors.LinearSegmentedColormap.from_list('plasmidis', np.vstack((viridis, plasma)))
if 'plasmidis' in mpl.colormaps:
    mpl.colormaps.unregister('plasmidis')
mpl.colormaps.register(plasmidis, name='plasmidis')

totalf = len(fileds)

In [ ]:
for k in range(len(fileds)):
    print(f'Processing {k+1}/{totalf}, {k+1/totalf*100:.2f}%')
    d = xr.open_dataset(fileds[k])
    p = xr.open_dataset(fileps[k])
    p.PhiDP.data = np.ma.masked_array(p.PhiDP.data, p.PhiDP.data <= -99900.0)
    p.PhiDP.data = np.rad2deg(p.PhiDP.data)
    r = xr.open_dataset(filers[k])
    v = xr.open_dataset(filevs[k])
    w = xr.open_dataset(filews[k])
    z = xr.open_dataset(filezs[k])

    all_fields = [{
        'varname' : 'Intensity',
        'data' : z,
        'label' : r'$Z$ dBZ',
        'cmap' : 'pyart_ChaseSpectral',
        'vmin' : -10,
        'vmax' : 80    
    }, {
        'varname' : 'Radial_Velocity',
        'data' : v,
        'label' : r'$V_{r}$ m/s',
        'cmap' : 'pyart_balance',
        'vmin' : -v.attrs['Nyquist_Vel-value'],
        'vmax' : v.attrs['Nyquist_Vel-value']
    }, {
        'varname' : 'Differential_Reflectivity',
        'data' : d,
        'label' : r'$Z_{DR}$ dB',
        'cmap' : 'pyart_RefDiff',
        'vmin' : -1,
        'vmax' : 8
    }, {
        'varname' : 'RhoHV',
        'data' : r,
        'label' : r'$\rho_{HV}$',
        'cmap' : 'plasmidis',
        'vmin' : 0,
        'vmax' : 1
    }, {
        'varname' : 'Width',
        'data' : w,
        'label' : 'Spectrum Width m/s',
        'cmap' : 'cubehelix',
        'vmin' : 0,
        'vmax' : 12.5
    }, {
        'varname' : 'PhiDP',
        'data' : p,
        'label' : r'$\phi_{DP}$ deg',
        'cmap' : 'pyart_Wild25',
        'vmin' : -180,
        'vmax' : 180
    }]


    px = 1/plt.rcParams['figure.dpi']
    fig, axs = plt.subplots(3, 2, subplot_kw={'projection': ccrs.LambertConformal()})
    idx = 0
    for i in range(3):
        for j in range(2):
            ax = axs[i, j]
            thisfield = all_fields[idx]
            data = thisfield['data']
            
            t_start = np.array([data.Time]).astype('datetime64[s]')[0]
            rng = np.matmul(data.Gate.data.reshape(-1, 1), data.GateWidth.data.reshape(1, -1))
            az = np.tile(data.Azimuth.data, (rng.shape[0], 1))
            el = np.tile(data.Elevation.data, (rng.shape[0], 1))
            rcs = RadarCoordinateSystem(data.Latitude, data.Longitude, 0)
            lon, lat, alt = rcs.toLonLatAlt(rng, az, el)
            lon = np.array(lon).reshape(az.shape)
            lat = np.array(lat).reshape(az.shape)
            alt = np.array(alt).reshape(az.shape)


            colorvar = np.ma.masked_array(data[thisfield['varname']].data, data[thisfield['varname']].data <= -99900.0).T

            handle = ax.pcolormesh(lon, lat, colorvar, vmin=thisfield['vmin'], vmax=thisfield['vmax'], cmap=thisfield['cmap'], transform=ccrs.PlateCarree())
            ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='gray', linewidth=0.5)
            fig.colorbar(handle, label=thisfield['label'])
            idx = idx+1
    time = pd.Timestamp(np.array([data.Time]).astype('datetime64[s]')[0]).to_pydatetime()
    fig.suptitle(f'RaXPol PPI\nAvg. Elevation={np.mean(el):.2f}\n'+time.strftime('%d %b %Y %H:%M:%S UTC'))
    fig.set_size_inches(1024*px, 1050*px)
    fig.tight_layout()
    fig.savefig(time.strftime('raxout/%Y%m%d_%H%M%S.png'))

from datetime import datetime as dt
import pandas as pd



ref = np.ma.masked_array(data.Intensity.data, data.Intensity.data < -100)



In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.LambertConformal())



In [20]:
import xarray as xr
import numpy as np
p = xr.open_dataset('rax/RAXPOL-20230616-005620-E2.5-P.nc')

In [21]:
p.PhiDP.data = np.ma.masked_array(p.PhiDP.data, p.PhiDP.data <= -100)
# p.PhiDP.data = np.rad2deg(p.PhiDP.data)
np.nanmin(p.PhiDP.data)

-3.1412072

In [ ]:
from datetime import datetime as dt
import pandas as pd

data = p

t_start = np.array([data.Time]).astype('datetime64[s]')[0]
range = np.matmul(data.Gate.data.reshape(-1, 1), data.GateWidth.data.reshape(1, -1))
az = np.tile(data.Azimuth.data, (range.shape[0], 1))
el = np.tile(data.Elevation.data, (range.shape[0], 1))
phidp = np.ma.masked_array(data.PhiDP.data, data.PhiDP.data < -100)

rcs = RadarCoordinateSystem(data.Latitude, data.Longitude, 0)
lon, lat, alt = rcs.toLonLatAlt(range, az, el)
lon = np.array(lon).reshape(az.shape)
lat = np.array(lat).reshape(az.shape)
alt = np.array(alt).reshape(az.shape)

d is zdr (Differential_Reflectivity)

p is phi (PhiDP)

r is cc (RhoHV)

v is velocity (Radial_Velocity)

w is width (Width)

z is reflectivity (Intensity)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection=ccrs.LambertConformal())
print(az.shape)
print(range.shape)
handle = ax.pcolormesh(lon, lat, ref.T, vmin=-10, vmax=80, cmap='pyart_ChaseSpectral', transform=ccrs.PlateCarree())
ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor='gray', linewidth=0.5)
fig.colorbar(handle)

In [ ]:
v

In [ ]:
v.attrs['Nyquist_Vel-value']